In [1]:
import pandas as pd
import numpy as np

Prepare Weather data

In [25]:
df_weather = pd.read_csv("data/weather_data_2022.csv")

In [26]:
display(df_weather)

,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2022-01-01,6:15 PM,33°F,28°F,82°%,NNE,21°mph,31°mph,29.17°in,0.0°in,Light Snow / Windy
1,2022-01-01,6:29 PM,32°F,27°F,82°%,NNE,22°mph,31°mph,29.17°in,0.0°in,Light Snow / Windy
2,2022-01-01,6:53 PM,32°F,27°F,82°%,NNE,21°mph,33°mph,29.18°in,0.0°in,Light Snow / Windy
3,2022-01-01,7:00 PM,32°F,28°F,85°%,NNE,22°mph,31°mph,29.18°in,0.0°in,Light Snow / Windy
4,2022-01-01,7:10 PM,32°F,28°F,85°%,NNE,21°mph,29°mph,29.17°in,0.0°in,Light Snow / Windy
...,...,...,...,...,...,...,...,...,...,...,...
10189,2022-12-31,8:53 PM,37°F,30°F,76°%,SE,9°mph,18°mph,29.23°in,0.0°in,Cloudy
10190,2022-12-31,9:53 PM,37°F,31°F,79°%,SE,10°mph,0°mph,29.21°in,0.0°in,Cloudy
10191,2022-12-31,10:53 PM,37°F,32°F,82°%,CALM,0°mph,0°mph,29.22°in,0.0°in,Cloudy
10192,2022-12-31,11:53 PM,37°F,32°F,82°%,SE,6°mph,0°mph,29.22°in,0.0°in,Cloudy


In [27]:
#create timestamp column and name it similar to the one of df_taxi in order to merge the two dataframes based on this column
df_weather['trip_start_timestamp'] = pd.to_datetime(df_weather['Date'] + ' ' + df_weather['Time'])

In [28]:
df_weather['Temperature'] = df_weather['Temperature'].str.extract('(\d+)').astype(float)
df_weather['Wind Speed'] = pd.to_numeric(df_weather['Wind Speed'].str.extract('(\d+)')[0], errors='coerce')
df_weather['Humidity'] = df_weather['Humidity'].str.replace(r'[^\d%]', '', regex=True).str.replace('%', '').astype(float) / 100
df_weather['Precip.'] = df_weather['Precip.'].replace('Trace', 0).replace(r'[^\d.]', '', regex=True).astype(float)

In [29]:
display(df_weather.dtypes)

Date                            object
Time                            object
Temperature                    float64
Dew Point                       object
Humidity                       float64
Wind                            object
Wind Speed                       int64
Wind Gust                       object
Pressure                        object
Precip.                        float64
Condition                       object
trip_start_timestamp    datetime64[ns]
dtype: object

In [30]:
df_weather = df_weather.sort_values('trip_start_timestamp')

In [31]:
display(df_weather)

,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,trip_start_timestamp
16,2022-01-01,12:13 AM,31.0,27°F,0.85,NE,21,28°mph,29.21°in,0.0,Light Snow / Windy,2022-01-01 00:13:00
17,2022-01-01,12:53 AM,32.0,26°F,0.79,NE,17,26°mph,29.23°in,0.0,Light Snow,2022-01-01 00:53:00
0,2022-01-01,6:15 PM,33.0,28°F,0.82,NNE,21,31°mph,29.17°in,0.0,Light Snow / Windy,2022-01-01 18:15:00
1,2022-01-01,6:29 PM,32.0,27°F,0.82,NNE,22,31°mph,29.17°in,0.0,Light Snow / Windy,2022-01-01 18:29:00
2,2022-01-01,6:53 PM,32.0,27°F,0.82,NNE,21,33°mph,29.18°in,0.0,Light Snow / Windy,2022-01-01 18:53:00
...,...,...,...,...,...,...,...,...,...,...,...,...
10188,2022-12-31,7:53 PM,36.0,30°F,0.79,VAR,3,0°mph,29.25°in,0.0,Cloudy,2022-12-31 19:53:00
10189,2022-12-31,8:53 PM,37.0,30°F,0.76,SE,9,18°mph,29.23°in,0.0,Cloudy,2022-12-31 20:53:00
10190,2022-12-31,9:53 PM,37.0,31°F,0.79,SE,10,0°mph,29.21°in,0.0,Cloudy,2022-12-31 21:53:00
10191,2022-12-31,10:53 PM,37.0,32°F,0.82,CALM,0,0°mph,29.22°in,0.0,Cloudy,2022-12-31 22:53:00


Prepare Taxi data

(include 'taxidata.csv' to data folder locally)

In [32]:
df_taxi = pd.read_parquet('data/prepped/prep_taxidata.parquet')

In [33]:
display(df_taxi.dtypes)

trip_id                               object
taxi_id                               object
trip_start_timestamp          datetime64[ns]
trip_end_timestamp            datetime64[ns]
trip_seconds                         float64
trip_miles                           float64
pickup_census_tract                  float64
dropoff_census_tract                 float64
fare                                 float64
tips                                 float64
tolls                                float64
extras                               float64
trip_total                           float64
payment_type                          object
company                               object
pickup_centroid_latitude             float64
pickup_centroid_longitude            float64
pickup_centroid_location              object
dropoff_centroid_latitude            float64
dropoff_centroid_longitude           float64
dropoff_centroid_location             object
trip_hours                           float64
1_hour_win

In [34]:
df_taxi = df_taxi.sort_values('trip_start_timestamp')

Merge both data frames based on 'trip_start_timestamp'

In [35]:
df_merged = pd.merge_asof(df_taxi, df_weather, on='trip_start_timestamp', direction='nearest')

In [36]:
display(df_merged)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,fare,tips,...,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,50719da0933d6056acca25c91c253012288c70c6,d9293712880e8a69bed187b718a3f5ef512c5263984006...,2022-01-01 00:00:00,2022-01-01 00:00:00,501.0,0.65,NaN,NaN,6.25,0.00,...,12:13 AM,31.0,27°F,0.85,NE,21,28°mph,29.21°in,0.0,Light Snow / Windy
1,5ad980003a4f7d9cd5264bbd38c65335d2d4354a,f1ed8794238deb1c8f7c3759aa096ae20e3e769cc4c9ad...,2022-01-01 00:00:00,2022-01-01 00:15:00,1330.0,12.92,NaN,NaN,33.00,0.00,...,12:13 AM,31.0,27°F,0.85,NE,21,28°mph,29.21°in,0.0,Light Snow / Windy
2,60efa1405c7dfbf8eecfaddb324cd323477b973d,42bfc19863617733aaa5ef96de670e3307690921a42d3d...,2022-01-01 00:00:00,2022-01-01 00:15:00,766.0,2.56,NaN,NaN,10.00,2.00,...,12:13 AM,31.0,27°F,0.85,NE,21,28°mph,29.21°in,0.0,Light Snow / Windy
3,603a126a15ae27b39c6e600581e90d95bbc8b945,261b4db1c52868b756351f3cba8d9cc18cbb0166b27610...,2022-01-01 00:00:00,2022-01-01 00:15:00,557.0,1.61,NaN,NaN,8.00,0.00,...,12:13 AM,31.0,27°F,0.85,NE,21,28°mph,29.21°in,0.0,Light Snow / Windy
4,5e9cb277072d99403f9e91eca8a448ea6f305be4,41a48c4607a84559d8760d511527018e3da952a7b33a82...,2022-01-01 00:00:00,2022-01-01 01:00:00,4006.0,15.30,NaN,NaN,47.50,0.00,...,12:13 AM,31.0,27°F,0.85,NE,21,28°mph,29.21°in,0.0,Light Snow / Windy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4739660,bba235de9bcc45331efbb3dfaf88a923b4ea0a5e,c57508f365f1bdac3be5e7de12fb19c8d669c37d515e41...,2022-12-31 23:45:00,2023-01-01 00:00:00,986.0,3.29,NaN,NaN,13.43,3.35,...,11:53 PM,37.0,32°F,0.82,SE,6,0°mph,29.22°in,0.0,Cloudy
4739661,c84619175dc673e7669c1c0578489742ee99e806,8e08799f421f71f00bb9ded6011b7e5369296773e81ad0...,2022-12-31 23:45:00,2023-01-01 00:00:00,640.0,2.98,NaN,NaN,11.00,0.00,...,11:53 PM,37.0,32°F,0.82,SE,6,0°mph,29.22°in,0.0,Cloudy
4739662,c9a54a709048dd0b7cf5d54cee92811450df1c96,4ba3690e7f7bde263c96de93e97af2af2545f6d16f06b7...,2022-12-31 23:45:00,2023-01-01 00:00:00,610.0,4.92,NaN,NaN,15.00,0.00,...,11:53 PM,37.0,32°F,0.82,SE,6,0°mph,29.22°in,0.0,Cloudy
4739663,697e35a751b59ca4ecf539ce367ea661737d9848,fb61583f0b732a39f08c04267cd6b923c05ccfb9baffb1...,2022-12-31 23:45:00,2022-12-31 23:45:00,337.0,0.74,NaN,NaN,5.75,0.00,...,11:53 PM,37.0,32°F,0.82,SE,6,0°mph,29.22°in,0.0,Cloudy


In [37]:
print(df_merged.isna().sum())

trip_id                             0
taxi_id                             0
trip_start_timestamp                0
trip_end_timestamp                  0
trip_seconds                        0
trip_miles                          0
pickup_census_tract           2574851
dropoff_census_tract          2574851
fare                                0
tips                                0
tolls                               0
extras                              0
trip_total                          0
payment_type                        0
company                             0
pickup_centroid_latitude            0
pickup_centroid_longitude           0
pickup_centroid_location            0
dropoff_centroid_latitude           0
dropoff_centroid_longitude          0
dropoff_centroid_location           0
trip_hours                          0
1_hour_window                       0
1_hour_window_sin                   0
1_hour_window_cos                   0
4_hour_window                       0
4_hour_windo

In [38]:
print(df_weather.isna().sum())

Date                    0
Time                    0
Temperature             0
Dew Point               0
Humidity                0
Wind                    6
Wind Speed              0
Wind Gust               0
Pressure                0
Precip.                 0
Condition               0
trip_start_timestamp    0
dtype: int64


For 2912 entries in the merged df the 'Wind', which is the wind direction, is missing. This results from only 6 missing values in the original df_weather (because one df_weather entry can be matched to multiple df_taxi entries). I suggest dropping the 'Wind' column, since we assume the wind direction not to be that relevant for our purposes.

In [39]:
df_merged = df_merged.drop('Wind',axis=1)

In [40]:
df_merged.to_parquet('data/prepped/weather_taxi_merged_df.parquet')

The resulting dataset is provided here: https://uni-koeln.sciebo.de/s/mCdIxvBvCsLZrth